In [1]:
%pip install --upgrade --quiet  langchain-google-genai langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
LANG_SMITH_API_KEY = userdata.get('LANGSMITH_KEY')

<font color="blue">TEST <b>GEMINI</b> FLASH 2.0 RUN</font> : <b>TESTRUN

In [4]:
### TEST GEMINI RUN
from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="How do i use langchain to trigger gemnini api call "
)
print(response.text)

```python
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import Runnable


# 1. Install Required Packages (if you haven't already)
#    pip install langchain langchain-google-genai

# 2. Set up your Google API Key
#    You'll need an API key for Google AI Studio.  You can get one here:
#    https://makersuite.google.com/app/apikey
#    Make sure to set it as an environment variable:
#    export GOOGLE_API_KEY="YOUR_API_KEY"
#
#    Alternatively, you can pass it directly in the ChatGoogleGenerativeAI constructor.
#    However, setting the environment variable is generally recommended for security reasons.

# 3. Initialize the Gemini Model
#   Choose the model you want to use. "gemini-pro" is a good general-purpose model.
llm = ChatGoogleGenerativeAI(model="gemini-pro")  # Set temperature to control randomness (0.0 is deterministic)

# OPTI

<font color="blue">GEMINI FLASH 2.0 using <b>LANGCHAIN</font> : Invocation

In [9]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    # os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your GEMINI API key: ")
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

# If you want to get automated tracing of your model calls you can also set your LangSmith API key by uncommenting below:
# os.environ["LANGSMITH_API_KEY"] = LANG_SMITH_API_KEY
# os.environ["LANGSMITH_TRACING"] = "true"

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1db75e2d-def4-47b8-8cfc-4c53bac64b70-0', usage_metadata={'input_tokens': 20, 'output_tokens': 7, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [15]:
print(ai_msg.content)

J'adore la programmation.


<font color="blue">GEMINI FLASH 2.0 using <b>LANGCHAIN</font> : Chaining</b> <br>

---
We can [chain](https://python.langchain.com/docs/how_to/sequence/) our model with a prompt template like so:

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain_output = chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)
chain_output

AIMessage(content='Ich liebe Programmieren.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-317a36ef-4b02-4bde-b7d3-e820db4e9590-0', usage_metadata={'input_tokens': 15, 'output_tokens': 7, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}})

In [21]:
print(chain_output.content)

Ich liebe Programmieren.


<font color="blue"> Safety Setting </font> : Experiment <br>

---
Gemini models have default safety settings that can be overridden. If you are receiving lots of "Safety Warnings" from your models, you can try tweaking the safety_settings attribute of the model. For example, to turn off safety blocking for dangerous content, you can construct your LLM as follows:




In [24]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [25]:
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), safety_settings={<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>}, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7d7b181e5690>, default_metadata=())

Refer :

---
Langchain : https://python.langchain.com/docs/integrations/chat/google_generative_ai/ <br>
Gemini API Dashboard : https://aistudio.google.com/apikey